In [113]:
import pandas as pd
import numpy as np

In [114]:
df = pd.read_excel('datasets\Bihar_Election_Results.xlsx')
df.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,2010,1,Rajesh Singh,42289,0.294,Janata Dal (United),Valmiki Nagar,1
1,2010,2,Mukesh Kumar Kushwaha,27618,0.192,Rashtriya Janata Dal,Valmiki Nagar,1
2,2010,3,Dheerendra Pratap Singh,20886,0.145,Bahujan Samaj Party,Valmiki Nagar,1
3,2010,4,Irshad Hussain,17747,0.124,Indian National Congress,Valmiki Nagar,1
4,2010,5,Deep Narayan Mahato,14047,0.098,Independent,Valmiki Nagar,1


In [115]:
df.describe()

,Position,Votes,Votes %,AC No
count,12301.000000,12301.000000,12301.000000,12301.000000
mean,7.690188,9290.666938,0.078570,122.960247
std,5.114163,16210.790784,0.129325,70.755933
min,1.000000,98.000000,0.001000,1.000000
25%,4.000000,969.000000,0.008000,61.000000
50%,7.000000,1975.000000,0.017000,125.000000
75%,11.000000,7022.000000,0.063000,186.000000
max,34.000000,117915.000000,0.721000,243.000000


In [116]:
df.isnull().any()

Election Year    False
Position         False
Name             False
Votes            False
Votes %          False
Party            False
AC name          False
AC No            False
dtype: bool

a. Determine the winning party for every election at the state level. A party which has the highest total votes is considered the winner here.

In [117]:
dfa = df[df['Position']==1].groupby(['Election Year','Party'])['AC No'].count().reset_index()
dfa = dfa.sort_values(['Election Year','AC No'],ascending=[True,False]).reset_index(drop=True)
dfa.rename(columns={'AC No':'Seats_Won'},inplace=True)
dfa['rank'] = dfa.groupby('Election Year')['Seats_Won'].rank(method='dense',ascending=False)
result = dfa[dfa['rank']==1].reset_index(drop=True)
result[['Election Year','Party','Seats_Won']]

,Election Year,Party,Seats_Won
0,2010,Janata Dal (United),115
1,2015,Rashtriya Janata Dal,80
2,2005-feb,Janata Dal (United),88
3,2005-oct,Rashtriya Janata Dal,75


b. determine the list of swing seats for the state. A swing seat is a constituency that has a different winner in every election. The elections are held in the order 2005-Feb, 2005-Oct, 2010, and 2015.

In [118]:
new_df = df.pivot_table(index='AC name', columns='Election Year', values='Party', aggfunc='first').reset_index(drop=False)
new_df = new_df[(new_df['2005-feb']!=new_df['2005-oct'])&(new_df['2005-oct']!=new_df[2010])&(new_df[2010]!=new_df[2015])]
swing = pd.DataFrame(new_df['AC name']).reset_index(drop=True)
swing


,AC name
0,Agiaon
1,Agiaon (SC)
2,Alauli
3,Alauli (SC)
4,Amour
...,...
108,Singheshwar (SC)
109,Sonbarsha (SC)
110,Tarari
111,Thakurganj


c. Determine the top 8 parties at the state-level in each election in terms of getting the most votes.

In [119]:
dfc = df.groupby(['Election Year','Party'])['Votes'].sum().reset_index()
dfc['rank']=dfc.groupby('Election Year')['Votes'].rank(method = 'dense', ascending = False)
dfc.sort_values(['Election Year','rank'], inplace = True)
dfc=dfc[dfc['rank']<=8].reset_index(drop = True)
dfc

,Election Year,Party,Votes,rank
0,2010,Janata Dal (United),6561906,1.0
1,2010,Rashtriya Janata Dal,5475656,2.0
2,2010,Bharatiya Janta Party,4790436,3.0
3,2010,Independent,3842812,4.0
4,2010,Indian National Congress,2431477,5.0
5,2010,Lok Jan Shakti Party,1957232,6.0
6,2010,Bahujan Samaj Party,933947,7.0
7,2010,Nationalist Congress Party,528575,8.0
8,2015,Bharatiya Janta Party,9308015,1.0
9,2015,Rashtriya Janata Dal,6995509,2.0


d. Determine the minimum votes that each party in the top 8 of the 2015 election secured among all the past elections. Do this for all constituencies.



In [120]:
dfd = dfc[dfc['Election Year']==2015]
dfd['min_vote']=df.groupby('Party')['Votes'].transform('min')
dfd

C:\Users\farzi\AppData\Local\Temp\ipykernel_25816\2423479204.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd['min_vote']=df.groupby('Party')['Votes'].transform('min')


,Election Year,Party,Votes,rank,min_vote
8,2015,Bharatiya Janta Party,9308015,1.0,98
9,2015,Rashtriya Janata Dal,6995509,2.0,255
10,2015,Janata Dal (United),6416414,3.0,98
11,2015,Independent,3580953,4.0,211
12,2015,Indian National Congress,2539638,5.0,98
13,2015,Lok Jan Shakti Party,1840834,6.0,98
14,2015,Rashtriya Lok Samta Party,976787,7.0,2432
15,2015,Hindustani Awam Morcha (Secular),864856,8.0,1100


e. Determine the consistently tri-polar constituencies. I.e Constituencies in which 3 parties or more got greater than 10% of the vote shares in each election. The parties can be different for different elections.

In [121]:
df.head(3)

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,2010,1,Rajesh Singh,42289,0.294,Janata Dal (United),Valmiki Nagar,1
1,2010,2,Mukesh Kumar Kushwaha,27618,0.192,Rashtriya Janata Dal,Valmiki Nagar,1
2,2010,3,Dheerendra Pratap Singh,20886,0.145,Bahujan Samaj Party,Valmiki Nagar,1


In [122]:
dfe= df[df['Votes %']>0.1].groupby(['Election Year','AC name'])['Party'].count().reset_index()
tri = dfe[dfe['Party']>=3].reset_index(drop=True)
tri_filt = tri.groupby('AC name')['Election Year'].count().reset_index()
tri_filt[tri_filt['Election Year']==3]['AC name'].values

array(['Agiaon', 'Alamnagar', 'Alinagar', 'Araria', 'Aurangabad',
       'Babubarhi', 'Bachhwara', 'Bahadurpur', 'Bakhri', 'Barari',
       'Baruraj', 'Begusarai', 'Belsand', 'Bhabua', 'Bihariganj',
       'Chakai', 'Cheria Bariarpur', 'Chiraia', 'Darauli', 'Dhaka',
       'Dhamdaha', 'Forbesganj', 'Gobindpur', 'Goh', 'Govindganj',
       'Jamalpur', 'Jokihat', 'Kahalgaon', 'Kalyanpur', 'Kesaria',
       'Kochadhaman', 'Kuchaikote', 'Lakhisarai', 'Lalganj', 'Mahishi',
       'Marhaura', 'Mohiuddinnagar', 'Narkatia', 'Nautan', 'Nawada',
       'Obra', 'Paliganj', 'Parihar', 'Raghunathpur', 'Raxaul', 'Riga',
       'Sandesh', 'Sherghati', 'Simri Bakhtiarpur', 'Siwan', 'Taraiya',
       'Tarapur', 'Teghra', 'Wazirganj', 'Ziradei'], dtype=object)

f. Determine the stronghold party-seats. A stronghold seat is one that is won by the same party in all the elections.

In [123]:
df.head(1)

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,2010,1,Rajesh Singh,42289,0.294,Janata Dal (United),Valmiki Nagar,1


In [124]:
dff = df[df['Position']==1].groupby('AC name')['Party'].count().reset_index()
dff[dff['Party']==1]['AC name'].values

array(['Agiaon (SC)', 'Alauli (SC)', 'Bakhri (SC)', 'Banmankhi (SC)',
       'Barachatti (SC)', 'Bathnaha (SC)', 'Bhorey (SC)', 'Bochaha (SC)',
       'Bodh Gaya (SC)', 'Chenari (SC)', 'Darauli (SC)', 'Dhauraiya (SC)',
       'Garkha (SC)', 'Harsidhi (SC)', 'Imamganj (SC)', 'Kalyanpur (SC)',
       'Katoria (ST)', 'Korha (SC)', 'Kusheshwar Asthan (SC)',
       'Kutumba (SC)', 'Makhadumapur (SC)', 'Manihari (ST)',
       'Masaurhi (SC)', 'Mohania (SC)', 'Patepur (SC)', 'Phulwari (SC)',
       'Pirpainti (SC)', 'Raja Pakar (SC)', 'Rajauli (SC)', 'Rajgir (SC)',
       'Rajnagar (SC)', 'Rajpur (SC)', 'Ramnagar (SC)', 'Raniganj (SC)',
       'Rosera (SC)', 'Runnisaidpur', 'Sakra (SC)', 'Sikandra (SC)',
       'Singheshwar (SC)', 'Sonbarsha (SC)', 'Triveniganj (SC)'],
      dtype=object)

g. Determine close contest constituencies. Close contest seats are those in which the top 2 finishers had a difference of less than 10% votes. Find seats in which there was a close contest in all the elections. If there is no such seat, then relax the criteria to 15%.

In [125]:
dfg = df[(df['Position']<=2)]
# dfg
dfg['Votes %'] = dfg.groupby(['Election Year','AC name'])['Votes %'].diff()
dfg.dropna(inplace=True)
dfg[abs(df['Votes %'])<=.15]['AC name'].values

C:\Users\farzi\AppData\Local\Temp\ipykernel_25816\3826895283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg['Votes %'] = dfg.groupby(['Election Year','AC name'])['Votes %'].diff()
C:\Users\farzi\AppData\Local\Temp\ipykernel_25816\3826895283.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg.dropna(inplace=True)
C:\Users\farzi\AppData\Local\Temp\ipykernel_25816\3826895283.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfg[abs(df['Votes %'])<=.15]['AC name'].values


array(['Bagaha', 'Bettiah', 'Jokihat', 'Dhamdaha', 'Korha', 'Alamnagar',
       'Bhorey', 'Lakhisarai', 'Kumhrarh', 'Manjhi', 'Chanpatia', 'Riga',
       'Hajipur', 'Samastipur', 'Teghra', 'Sahebpur Kamal', 'Bankipur',
       'Chainpur', 'Jamui'], dtype=object)

h. Find the growing seats. For the top 8 parties at the state-level in the 2005-Feb Election, Determine the seats in which these parties have continuously increased their votes in the next 3 elections.

In [126]:
dfh = df.pivot_table(index = 'AC name',columns='Election Year',values='Votes',aggfunc='first').reset_index()
dfh[(dfh['2005-feb'] < dfh['2005-oct'])&(dfh['2005-oct']<dfh[2010])&(dfh[2010]<dfh[2015])]['AC name'].values

array(['Alamnagar', 'Amarpur', 'Amour', 'Araria', 'Arrah', 'Aurangabad',
       'Bankipur', 'Baruraj', 'Buxar', 'Chainpur', 'Chapra', 'Chhatapur',
       'Danapur', 'Forbesganj', 'Gaighat', 'Gaya Town', 'Goh',
       'Gopalganj', 'Gopalpur', 'Gurua', 'Islampur', 'Karakat', 'Kasba',
       'Keoti', 'Kochadhaman', 'Madhubani', 'Matihani', 'Motihari',
       'Munger', 'Muzaffarpur', 'Narpatganj', 'Nawada', 'Parbatta',
       'Purnia', 'Saharsa', 'Sarairanjan', 'Sheohar', 'Tikari',
       'Vaishali', 'Valmiki Nagar'], dtype=object)

i. Find the declining seats. Repeat exercise g for the opposite growth.

In [127]:
dfi = df.pivot_table(index = 'AC name',columns='Election Year',values='Votes',aggfunc='first').reset_index()
dfi[(dfi['2005-feb'] > dfi['2005-oct'])&(dfi['2005-oct']>dfi[2010])&(dfi[2010]>dfi[2015])]['AC name'].values

array([], dtype=object)

j. Determine the state-winner-match constituencies. Find the seats which have always elected the party that has gone on to win the state election (most votes secured at state). So if the RJD won at the state level in 2010, then seats which voted an RJD candidate as winner are considered in this category. This criteria for a seat has to be matched for each election.

In [128]:
df.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,2010,1,Rajesh Singh,42289,0.294,Janata Dal (United),Valmiki Nagar,1
1,2010,2,Mukesh Kumar Kushwaha,27618,0.192,Rashtriya Janata Dal,Valmiki Nagar,1
2,2010,3,Dheerendra Pratap Singh,20886,0.145,Bahujan Samaj Party,Valmiki Nagar,1
3,2010,4,Irshad Hussain,17747,0.124,Indian National Congress,Valmiki Nagar,1
4,2010,5,Deep Narayan Mahato,14047,0.098,Independent,Valmiki Nagar,1


In [129]:
dfj = df[df['Position']==1].groupby(['Election Year','Party'])['Party'].count().reset_index(name='count')
dfj.sort_values(by=['Election Year','count'],ascending=False,inplace=True)
dfj['rank'] = dfj.groupby('Election Year')['count'].rank(method = 'dense',ascending=False)
dfj = dfj[dfj['rank']==1].reset_index()
dfj

,index,Election Year,Party,count,rank
0,40,2005-oct,Rashtriya Janata Dal,75,1.0
1,25,2005-feb,Janata Dal (United),88,1.0
2,15,2015,Rashtriya Janata Dal,80,1.0
3,4,2010,Janata Dal (United),115,1.0


In [163]:
y1 = set(df[(df['Position']==1)&(df['Party']=='Rashtriya Janata Dal')]['AC name'])
y1

{'Alamnagar',
 'Alauli',
 'Alinagar',
 'Araria',
 'Arrah',
 'Arwal',
 'Atri',
 'Aurai',
 'Babubarhi',
 'Bachhwara',
 'Bahadurpur',
 'Baisi',
 'Bajpatti',
 'Bakhri',
 'Bakhri (SC)',
 'Bakhtiarpur',
 'Balrampur',
 'Baniapur',
 'Banka',
 'Barachatti',
 'Barari',
 'Barauli',
 'Barbigha',
 'Barhampur',
 'Barhara',
 'Barharia',
 'Baruraj',
 'Begusarai',
 'Belaganj',
 'Benipatti',
 'Bhagalpur',
 'Bhorey (SC)',
 'Bihariganj',
 'Biharsharif',
 'Bihpur',
 'Bikram',
 'Bisfi',
 'Bochaha',
 'Bochaha (SC)',
 'Bodh Gaya',
 'Bodh Gaya (SC)',
 'Brahampur',
 'Buxar',
 'Chakai',
 'Darbhanga',
 'Darbhanga Rural',
 'Dehri',
 'Dhaka',
 'Dhamdaha',
 'Dinara',
 'Fatuha',
 'Fatwah',
 'Gaighat',
 'Garkha',
 'Gaura Bauram',
 'Ghosi',
 'Gobindpur',
 'Goriyakothi',
 'Harsidhi',
 'Hayaghat',
 'Hilsa',
 'Hisua',
 'Imamganj',
 'Imamganj (SC)',
 'Jagdishpur',
 'Jahanabad',
 'Jale',
 'Jamui',
 'Jhanjharpur',
 'Jokihat',
 'Kadwa',
 'Kalyanpur',
 'Karakat',
 'Katihar',
 'Katoria',
 'Katoria (ST)',
 'Keoti',
 'Kesaria',
 

In [164]:
y2 = set(df[(df['Position']==1)&(df['Party']=='Janata Dal (United)')]['AC name'])
y2

{'Agiaon',
 'Alamnagar',
 'Alauli',
 'Amarpur',
 'Amnour',
 'Amour',
 'Asthawan',
 'Atri',
 'Aurangabad',
 'Babubarhi',
 'Bagaha',
 'Bahadurganj',
 'Bahadurpur',
 'Baikunthpur',
 'Baisi',
 'Bajpatti',
 'Bakhtiarpur',
 'Baniapur',
 'Bankipur',
 'Barachatti',
 'Barbigha',
 'Barh',
 'Barhara',
 'Barharia',
 'Belaganj',
 'Beldaur',
 'Belhar',
 'Belsand',
 'Benipatti',
 'Benipur',
 'Bhabua',
 'Bhorey',
 'Bibhutpur',
 'Bihariganj',
 'Biharsharif',
 'Bisfi',
 'Bochaha',
 'Chapra',
 'Chenari',
 'Cheria Bariarpur',
 'Chhatapur',
 'Chiraia',
 'Danapur',
 'Daraundha',
 'Darbhanga Rural',
 'Dhamdaha',
 'Dhauraiya (SC)',
 'Dhuraiya',
 'Digha',
 'Dinara',
 'Dumraon',
 'Ekma',
 'Garkha',
 'Garkha (SC)',
 'Gaya Town',
 'Ghosi',
 'Gobindpur',
 'Goh',
 'Gopalpur',
 'Gora Bauram',
 'Goriakothi',
 'Govindganj',
 'Gurua',
 'Hajipur',
 'Harlakhi',
 'Harnaut',
 'Harsidhi',
 'Harsidhi (SC)',
 'Hasanpur',
 'Hathua',
 'Hayaghat',
 'Hilsa',
 'Hisua',
 'Imamganj',
 'Islampur',
 'Jagdishpur',
 'Jahanabad',
 'Jamal

In [165]:
y3 = set(df[(df['Position']==1)&(df['Party']=='Rashtriya Janata Dal')]['AC name'])
y3

{'Alamnagar',
 'Alauli',
 'Alinagar',
 'Araria',
 'Arrah',
 'Arwal',
 'Atri',
 'Aurai',
 'Babubarhi',
 'Bachhwara',
 'Bahadurpur',
 'Baisi',
 'Bajpatti',
 'Bakhri',
 'Bakhri (SC)',
 'Bakhtiarpur',
 'Balrampur',
 'Baniapur',
 'Banka',
 'Barachatti',
 'Barari',
 'Barauli',
 'Barbigha',
 'Barhampur',
 'Barhara',
 'Barharia',
 'Baruraj',
 'Begusarai',
 'Belaganj',
 'Benipatti',
 'Bhagalpur',
 'Bhorey (SC)',
 'Bihariganj',
 'Biharsharif',
 'Bihpur',
 'Bikram',
 'Bisfi',
 'Bochaha',
 'Bochaha (SC)',
 'Bodh Gaya',
 'Bodh Gaya (SC)',
 'Brahampur',
 'Buxar',
 'Chakai',
 'Darbhanga',
 'Darbhanga Rural',
 'Dehri',
 'Dhaka',
 'Dhamdaha',
 'Dinara',
 'Fatuha',
 'Fatwah',
 'Gaighat',
 'Garkha',
 'Gaura Bauram',
 'Ghosi',
 'Gobindpur',
 'Goriyakothi',
 'Harsidhi',
 'Hayaghat',
 'Hilsa',
 'Hisua',
 'Imamganj',
 'Imamganj (SC)',
 'Jagdishpur',
 'Jahanabad',
 'Jale',
 'Jamui',
 'Jhanjharpur',
 'Jokihat',
 'Kadwa',
 'Kalyanpur',
 'Karakat',
 'Katihar',
 'Katoria',
 'Katoria (ST)',
 'Keoti',
 'Kesaria',
 

In [166]:
y4 = set(df[(df['Position']==1)&(df['Party']=='Janata Dal (United)')]['AC name'])
y4

{'Agiaon',
 'Alamnagar',
 'Alauli',
 'Amarpur',
 'Amnour',
 'Amour',
 'Asthawan',
 'Atri',
 'Aurangabad',
 'Babubarhi',
 'Bagaha',
 'Bahadurganj',
 'Bahadurpur',
 'Baikunthpur',
 'Baisi',
 'Bajpatti',
 'Bakhtiarpur',
 'Baniapur',
 'Bankipur',
 'Barachatti',
 'Barbigha',
 'Barh',
 'Barhara',
 'Barharia',
 'Belaganj',
 'Beldaur',
 'Belhar',
 'Belsand',
 'Benipatti',
 'Benipur',
 'Bhabua',
 'Bhorey',
 'Bibhutpur',
 'Bihariganj',
 'Biharsharif',
 'Bisfi',
 'Bochaha',
 'Chapra',
 'Chenari',
 'Cheria Bariarpur',
 'Chhatapur',
 'Chiraia',
 'Danapur',
 'Daraundha',
 'Darbhanga Rural',
 'Dhamdaha',
 'Dhauraiya (SC)',
 'Dhuraiya',
 'Digha',
 'Dinara',
 'Dumraon',
 'Ekma',
 'Garkha',
 'Garkha (SC)',
 'Gaya Town',
 'Ghosi',
 'Gobindpur',
 'Goh',
 'Gopalpur',
 'Gora Bauram',
 'Goriakothi',
 'Govindganj',
 'Gurua',
 'Hajipur',
 'Harlakhi',
 'Harnaut',
 'Harsidhi',
 'Harsidhi (SC)',
 'Hasanpur',
 'Hathua',
 'Hayaghat',
 'Hilsa',
 'Hisua',
 'Imamganj',
 'Islampur',
 'Jagdishpur',
 'Jahanabad',
 'Jamal

In [177]:
# len(list(y1&y2&y3&y4))
list(y1&y2&y3&y4)

['Parbatta',
 'Imamganj',
 'Lalganj',
 'Nathnagar',
 'Sahebpur Kamal',
 'Madhubani',
 'Khajauli',
 'Sasaram',
 'Valmiki Nagar',
 'Jhanjharpur',
 'Barbigha',
 'Sonepur',
 'Suryagarha',
 'Hayaghat',
 'Sahebganj',
 'Jahanabad',
 'Kochadhaman',
 'Barhara',
 'Bochaha',
 'Minapur',
 'Katoria',
 'Kutumba',
 'Nawada',
 'Taraiya',
 'Kalyanpur',
 'Hilsa',
 'Jagdishpur',
 'Narkatia',
 'Raja Pakar',
 'Harsidhi',
 'Bajpatti',
 'Tarari',
 'Munger',
 'Atri',
 'Babubarhi',
 'Bisfi',
 'Morwa',
 'Masaurhi',
 'Madhuban',
 'Nokha',
 'Hisua',
 'Mahishi',
 'Barachatti',
 'Maner',
 'Thakurganj',
 'Bakhtiarpur',
 'Tribeniganj',
 'Gobindpur',
 'Makhdumpur',
 'Jokihat',
 'Nabinagar',
 'Biharsharif',
 'Sherghati',
 'Vaishali',
 'Kadwa',
 'Belaganj',
 'Parsa',
 'Alamnagar',
 'Benipatti',
 'Madhepura',
 'Dinara',
 'Supaul',
 'Kurhani',
 'Sursand',
 'Raghopur',
 'Alauli',
 'Sheohar',
 'Katihar',
 'Pipra',
 'Dhamdaha',
 'Rajnagar',
 'Karakat',
 'Kuchaikote',
 'Bihariganj',
 'Ghosi',
 'Barharia',
 'Darbhanga Rural',
